In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install einops
%pip install wandb

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 163kB 51.3MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 133kB 56.1MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=cf21a770b545fd1233ccc9ec7a91255b78978bd05183e2c09d9f084bd95c96f8
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=7e0290236ea98122ca6927d45e106d3ec50eb3285e2dcc0c5961751bbb4ae0d5
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [4]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [10]:
from model.CvT import CvT
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [6]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [7]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 32,
    "patch_size" : 16,
    "num_classes" : 10,
    "dim" : 128,
    "depth" : 12,
    "heads" : 8,
    "mlp_dim" : 1024,
    "channels" : 3,
    "dropout" : 0.1,
    "emb_dropout": 0.
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar10_ViT", entity='NoguNogu',name='CvT')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
trainloader, testloader = set_transforms(
    config['image_size'],
    config['batch_size'],
    name='Cifar10')._set_transforms()

In [11]:
model = CvT(
    num_classes= config['num_classes']
    ).to(device)

In [14]:
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 8, 8]           9,472
         LayerNorm-2             [-1, 8, 8, 64]             128
            Conv2d-3             [-1, 64, 8, 8]             576
       BatchNorm2d-4             [-1, 64, 8, 8]             128
            Conv2d-5             [-1, 64, 8, 8]           4,096
   DepthWiseConv2d-6             [-1, 64, 8, 8]               0
            Conv2d-7             [-1, 64, 4, 4]             576
       BatchNorm2d-8             [-1, 64, 4, 4]             128
            Conv2d-9            [-1, 128, 4, 4]           8,192
  DepthWiseConv2d-10            [-1, 128, 4, 4]               0
          Softmax-11               [-1, 64, 16]               0
           Conv2d-12             [-1, 64, 8, 8]           4,160
          Dropout-13             [-1, 64, 8, 8]               0
        Attention-14             [-1, 6

In [15]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [16]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 1.9096 - acc: 0.3033 - val_loss : 1.8027 - val_acc: 0.3366




Epoch : 2 - loss : 1.7704 - acc: 0.3539 - val_loss : 1.7245 - val_acc: 0.3726




Epoch : 3 - loss : 1.7011 - acc: 0.3822 - val_loss : 1.6752 - val_acc: 0.3882




Epoch : 4 - loss : 1.6562 - acc: 0.4003 - val_loss : 1.6470 - val_acc: 0.3987




Epoch : 5 - loss : 1.6235 - acc: 0.4133 - val_loss : 1.6138 - val_acc: 0.4118



...


Epoch : 16 - loss : 1.4273 - acc: 0.4868 - val_loss : 1.4364 - val_acc: 0.4903




Epoch : 17 - loss : 1.4106 - acc: 0.4929 - val_loss : 1.4454 - val_acc: 0.4760




Epoch : 18 - loss : 1.4008 - acc: 0.4983 - val_loss : 1.4329 - val_acc: 0.4812




Epoch : 19 - loss : 1.3885 - acc: 0.5047 - val_loss : 1.4130 - val_acc: 0.4975




Epoch : 20 - loss : 1.3797 - acc: 0.5072 - val_loss : 1.4233 - val_acc: 0.4982



In [18]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  6 11:39:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    34W / 250W |   1857MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------